In [ ]:
from math import log2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random


In [ ]:
# proc

def get_entropy(P1,P2):
    E1 = 0 if P1 == 0 else (-P1*log2(P1))
    E2 = 0 if P2 == 0 else (-P2*log2(P2))
    return (E1 + E2)

def get_gain(x, y, split):      
    num_gt_splt = len(x[x>=split])
    num_ls_splt = len(x[x<split])   
    num_label1 = len(y[y==1])
    num_label0 = len(y[y==0])

    pos1 = num_label1/len(y)
    pos0 = num_label0/len(y)
    pos_gt = num_gt_splt/len(x)
    pos_ls = num_ls_splt/len(x)
    pos_label1_gt = 0 if (num_gt_splt == 0) else len(y[(y==1) & (x>=split)])/num_gt_splt
    pos_label0_gt = 0 if (num_gt_splt == 0) else len(y[(y==0) & (x>=split)])/num_gt_splt
    pos_label1_ls = 0 if (num_ls_splt == 0) else len(y[(y==1) & (x<split)])/num_ls_splt
    pos_label0_ls = 0 if (num_ls_splt == 0) else len(y[(y==0) & (x<split)])/num_ls_splt
    
    H_gt = get_entropy(pos_label1_gt, pos_label0_gt)
    H_ls = get_entropy(pos_label1_ls, pos_label0_ls)
    H_cond = H_gt*pos_gt+H_ls*pos_ls
    H_y = get_entropy(pos1, pos0)
    infogain = H_y - H_cond
    gain_ratio = 0 if (get_entropy(pos_gt,pos_ls) == 0) else infogain/get_entropy(pos_gt,pos_ls)
    return infogain, gain_ratio

def find_split(x, y):
    found_split = 0
    found_ratio = 0
    found_idx = 0
    for feature_idx in range(0,x.shape[1]):     #iterate all features 
        features = x.iloc[:,feature_idx]
        for feature in features:
            infogain,gain_ratio = get_gain(features, y, feature)
            if gain_ratio > found_ratio:
                found_ratio = gain_ratio
                found_split = feature
                found_idx = feature_idx
    #print("X{} >= {}".format(found_idx, found_split) )
    return found_ratio, found_split, found_idx

def split_traindata(data, rd_idx, num):
    if(num >8192):
        print("train too large")
        raise SystemExit()
    data_train = data.loc[rd_idx[:num]]
    x_train = data_train.iloc[:,:-1]
    y_train = data_train.iloc[:,-1]
    return x_train, y_train

def rate_error(T, x, y):
    res_arr = T.array_predict(x)
    return (res_arr!=y).sum()/len(y)


In [ ]:
#class

class Node:
    def __init__(self, x, y, depth=0):
        self.leftnode = None
        self.rightnode = None
        self.depth = depth
        self.features = x
        self.label = y
        self.gain_ratio, self.split, self.split_idx = find_split(x,y)
        #print("--its DEPTH : ",depth," --\n")
        self.prediction = 1 if (len(y[y==1]) > len(y[y==0])) else 0
    
    def is_leaf(self):   
        if len(self.features) == 0 or len(self.features) == 1: 
            return 1
        elif self.gain_ratio == 0:
            return 1
        else:
            return 0

class DecisionTree:
    def __init__(self):
        self.root = None
        self.node_num = 0
        
    def grow(self,x,y,depth = 0):
        node = Node(x,y,depth)
        if node.is_leaf() != 1: 
            self.node_num += 1    
            x_left = x[x.iloc[:,node.split_idx]>=node.split]
            x_right = x[x.iloc[:,node.split_idx]<node.split]
            y_left = y[x.iloc[:,node.split_idx]>=node.split]
            y_right = y[x.iloc[:,node.split_idx]<node.split]
            node.leftnode = self.grow(x_left, y_left, node.depth+1)
            node.rightnode = self.grow(x_right, y_right, node.depth+1)
        self.root = node
        return node

    def predict(self, x):       #input one item and get prediction
        if self.root == None:
            print("Pls Grow a Tree before prediction!")
            raise SystemExit()
            return -1
        node = self.root
        while node.is_leaf() != 1:
            if x[node.split_idx] >= node.split:
                node = node.leftnode
            else:
                node = node.rightnode
        return node.prediction  
    
    def array_predict(self, feature_arr):
        res_list = []
        if self.root == None:
            print("Pls Grow a Tree before prediction!")
            raise SystemExit()
            return -1
        for i in range(feature_arr.shape[0]):
            res_list.append(self.predict(feature_arr.iloc[i,:]))
        return np.array(res_list)
    
    def disp(self,rootnode,lr = ""):
        if self.root == None:
            print("The tree is empty!")
            raise SystemExit()
            return -1
        node = rootnode
        for i in range(node.depth):
            print("+---",end="  ")
        if node.is_leaf() != 1:
            print(lr,"Dec Node X{} >= {}".format(node.split_idx, node.split))
            self.disp(node.leftnode,"left[X{}>={}]:".format(node.split_idx,node.split))
            self.disp(node.rightnode,"right[X{}<{}]:".format(node.split_idx,node.split))
        else:
            print(lr,"Leave(label ={})".format(node.prediction))
            return node
        
    def plot_boundary(self,x,y,low=0, high=1):
        x0grid = np.arange(low,high,0.01)
        x1grid = np.arange(low,high,0.01)
        xx,yy = np.meshgrid(x0grid, x1grid)
        x0, x1 = xx.flatten().reshape((len(xx)*xx.shape[1], 1)), yy.flatten().reshape((len(xx)*xx.shape[1], 1))
        
        #full coordinate samples
        grid = pd.DataFrame(np.hstack((x0,x1)))
        predict_arr = self.array_predict(grid)  # get all prediction
        zz = predict_arr.reshape(xx.shape)
        
        # scatter the training items
        plt.scatter(x.iloc[:,0][y==1], x.iloc[:,1][y==1])
        plt.scatter(x.iloc[:,0][y==0], x.iloc[:,1][y==0])
        # cover full coordinate
        plt.contourf(xx,yy,zz, alpha = 0.5)
        plt.xlabel('x0')
        plt.ylabel('x1')
        return
                  

In [ ]:
# Q3

data_q3 = pd.read_table('../data/Druns.txt',sep = ' ', header=None)
x = data_q3.iloc[:,:-1]
y = data_q3.iloc[:,-1]

print("SPLIT\tINFOGAIN\t\tGAINRATIO")
print("<<Feature X1>>")
for i in range(x.shape[0]):
    infogain,gain_ratio = get_gain(x.iloc[:,1],y,x.iloc[i,1])
    print(">={}\t{}\t{}".format(x.iloc[i,1], infogain, gain_ratio))
print("<<Feature X0>>")
for i in range(x.shape[0]):
    infogain,gain_ratio = get_gain(x.iloc[:,0],y,x.iloc[i,0])
    print(">={}\t{}\t{}".format(x.iloc[i,0], infogain, gain_ratio))

In [ ]:
#Q4

data_q4 = pd.read_table('../data/D3leaves.txt',sep = ' ', header=None)
x = data_q4.iloc[:,:-1]
y = data_q4.iloc[:,-1]
T = DecisionTree()
T.grow(x,y)
T.disp(T.root)

In [ ]:
#Q5

data_q5 = pd.read_table('../data/D1.txt',sep = ' ', header=None)
data_q5 = pd.read_table('../data/D2.txt',sep = ' ', header=None)
x = data_q5.iloc[:,:-1]
y = data_q5.iloc[:,-1]
T = DecisionTree()
T.grow(x,y)
T.disp(T.root)

In [ ]:
#Q6

data_q6 = pd.read_table('../data/D1.txt',sep = ' ', header=None)
x = data_q6.iloc[:,:-1]
y = data_q6.iloc[:,-1]
T = DecisionTree()
T.grow(x,y)
plt.figure(1,figsize=(10, 6))
plt.subplot(1,2,1)
T.plot_boundary(x,y)
plt.title("D1.txt 's boundry")
plt.subplot(1,2,2)
data_q6 = pd.read_table('../data/D2.txt',sep = ' ', header=None)
x = data_q6.iloc[:,:-1]
y = data_q6.iloc[:,-1]
T = DecisionTree()
T.grow(x,y)
T.plot_boundary(x,y)
plt.title("D2.txt 's boundry")

In [ ]:
#Q7
file = "../data/Dbig.txt"
data_q7 = pd.read_table(file,sep = ' ', header=None)
x = data_q7.iloc[:,:-1]
y = data_q7.iloc[:,-1]
#random permutation
random_idx = np.random.RandomState(seed=6).permutation(10000)
error_arr = []
node_arr = []
#1808 fixed test set
x_test = data_q7.iloc[random_idx[8192:],:-1]
y_test = data_q7.iloc[random_idx[8192:],-1]


plt.figure(2,figsize=(15, 10))
#-------n = 32-------#
x_train, y_train = split_traindata(data_q7, random_idx, 32)
T = DecisionTree()
T.grow(x_train,y_train)
plt.subplot(2,3,1)
T.plot_boundary(x, y, -1.5, 1.5)
error_arr.append(rate_error(T, x_test, y_test))
node_arr.append(T.node_num)
plt.title("n = 32")

#-------n = 128-------#
x_train, y_train = split_traindata(data_q7, random_idx, 128)
T = DecisionTree()
T.grow(x_train,y_train)
plt.subplot(2,3,2)
T.plot_boundary(x, y, -1.5, 1.5)
error_arr.append(rate_error(T, x_test, y_test))
node_arr.append(T.node_num)
plt.title("n = 128")

#-------n = 512-------#
x_train, y_train = split_traindata(data_q7, random_idx, 512)
T = DecisionTree()
T.grow(x_train,y_train)
plt.subplot(2,3,3)
T.plot_boundary(x, y, -1.5, 1.5)
error_arr.append(rate_error(T, x_test, y_test))
node_arr.append(T.node_num)
plt.title("n = 512")

#-------n = 2048-------#
x_train, y_train = split_traindata(data_q7, random_idx, 2048)
T = DecisionTree()
T.grow(x_train,y_train)
plt.subplot(2,3,4)
T.plot_boundary(x, y, -1.5, 1.5)
error_arr.append(rate_error(T, x_test, y_test))
node_arr.append(T.node_num)
plt.title("n = 2048")

#-------n = 8192-------#
x_train, y_train = split_traindata(data_q7, random_idx, 8192)
T = DecisionTree()
T.grow(x_train,y_train)
plt.subplot(2,3,5)
T.plot_boundary(x, y, -1.5, 1.5)
error_arr.append(rate_error(T, x_test, y_test))
node_arr.append(T.node_num)
plt.title("n = 8192")

print(error_arr,node_arr)
set_list = [32,128,512,2048,8192]
plt.figure(3)
plt.plot(set_list, error_arr)
plt.xlabel("number of train set")
plt.ylabel("error rate")